In [119]:
import requests
import bs4
import pandas as pd
from datetime import datetime
import time
from math import floor

In [56]:
year = 2019
url = 'https://www.baseball-reference.com/leagues/MLB/' + str(year) + '-schedule.shtml'

In [57]:
res = requests.get(url)
soup=bs4.BeautifulSoup(res.text, 'html.parser')

In [58]:
res

<Response [200]>

In [59]:
content = soup.find('div',{'class': "section_content"})

In [7]:
len(content('div'))

185

In [48]:
YEAR = []
DATES_ENGLISH = []
DATES_ALPHA = []
AWAYTEAMFULL = []
AWAYTEAMABRR = []
HOMETEAMFULL = []
HOMETEAMABRR = []
AWAYTEAMSCORE = []
HOMETEAMSCORE = []

In [122]:
start = time.perf_counter()

YEAR = []
DATES_ENGLISH = []
DATES_ALPHA = []
AWAYTEAMFULL = []
AWAYTEAMABRR = []
HOMETEAMFULL = []
HOMETEAMABRR = []
AWAYTEAMSCORE = []
HOMETEAMSCORE = []
STARTIME = []
ATTENDANCE = []
VENUE = []
DURATION = []

for divs in content('div')[:5]:
    h = divs.find('h3')
    games = divs('p', {'class':'game'})
    for p in games:
        a1 = p('a')[0]
        a2 = p('a')[1]
        
        boxscore = p.find('em')
        boxUrl = 'https://www.baseball-reference.com/' + boxscore.find('a')['href']
        
        boxRes = requests.get(boxUrl)
        boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')
        
        boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})
        
        for bm in boxMeta('div'):
            for bms in bm('strong'):

                if bms.text.strip() == 'Attendance':
                    attendance = int(bm.text.split(': ')[1].replace(',',''))
                elif bms.text.strip() == 'Venue':
                    venue = bm.text.split(': ')[1]
                elif bms.text.strip() == 'Game Duration':
                    gameDur = bm.text.split(': ')[1]
                    
        YEAR.append(year)
        DATES_ENGLISH.append(h.text)
        DATES_ALPHA.append(datetime.strptime(h.text, '%A, %B %d, %Y').strftime("%Y-%m-%d"))
        AWAYTEAMFULL.append(a1.text)
        HOMETEAMFULL.append(a2.text)
        AWAYTEAMABRR.append(a1['href'].split('/')[2])
        HOMETEAMABRR.append(a2['href'].split('/')[2])
        AWAYTEAMSCORE.append(int(p.text.split('(')[1].split(')')[0]))
        HOMETEAMSCORE.append(int(p.text.split('(')[2].split(')')[0]))
        STARTIME.append(boxMeta('div')[1].text.split(': ')[1])
        ATTENDANCE.append(attendance)
        VENUE.append(venue)
        DURATION.append(gameDur)

        elapsed_ = time.perf_counter() - start
        hours   = floor(elapsed_ / 60**2)
        minutes = floor((elapsed_ - hours*60**2) / 60)
        seconds = floor(elapsed_ - hours*60**2 -minutes*60)
        
        print(h.text, ' - ', a1['href'].split('/')[2] + '-' + a2['href'].split('/')[2], ' - ',  'Elapsed Time:',  hours, 'hours', minutes, 'minutes', seconds, 'seconds')
    

columns = ['year', 'dateEng', 'date', 'startTime', 'awayTeam', 'awayTeamAbbr', 'homeTeam', 'homeTeamAbbr', 'awayTeamScore', 'homeTeamScore', 'venue', 'attendance', 'gameDuration']
dataDict = {}
dataDict['year'] = YEAR
dataDict['dateEng'] = DATES_ENGLISH
dataDict['date'] = DATES_ALPHA
dataDict['awayTeam'] = AWAYTEAMFULL
dataDict['awayTeamAbbr'] = AWAYTEAMABRR
dataDict['homeTeam'] = HOMETEAMFULL
dataDict['homeTeamAbbr'] = HOMETEAMABRR
dataDict['awayTeamScore'] = AWAYTEAMSCORE
dataDict['homeTeamScore'] = HOMETEAMSCORE
dataDict['startTime'] = STARTIME
dataDict['attendance'] = ATTENDANCE
dataDict['venue'] = VENUE
dataDict['gameDuration'] = DURATION

df = pd.DataFrame(dataDict, columns=columns)
df

Wednesday, March 20, 2019  -  SEA-OAK  -  Elapsed Time: 0 hours 0 minutes 0 seconds
Thursday, March 21, 2019  -  SEA-OAK  -  Elapsed Time: 0 hours 0 minutes 0 seconds
Thursday, March 28, 2019  -  PIT-CIN  -  Elapsed Time: 0 hours 0 minutes 1 seconds
Thursday, March 28, 2019  -  CHW-KCR  -  Elapsed Time: 0 hours 0 minutes 2 seconds
Thursday, March 28, 2019  -  ARI-LAD  -  Elapsed Time: 0 hours 0 minutes 2 seconds
Thursday, March 28, 2019  -  COL-MIA  -  Elapsed Time: 0 hours 0 minutes 3 seconds
Thursday, March 28, 2019  -  STL-MIL  -  Elapsed Time: 0 hours 0 minutes 3 seconds
Thursday, March 28, 2019  -  CLE-MIN  -  Elapsed Time: 0 hours 0 minutes 4 seconds
Thursday, March 28, 2019  -  BAL-NYY  -  Elapsed Time: 0 hours 0 minutes 4 seconds
Thursday, March 28, 2019  -  LAA-OAK  -  Elapsed Time: 0 hours 0 minutes 5 seconds
Thursday, March 28, 2019  -  ATL-PHI  -  Elapsed Time: 0 hours 0 minutes 5 seconds
Thursday, March 28, 2019  -  SFG-SDP  -  Elapsed Time: 0 hours 0 minutes 6 seconds
Thu

,year,dateEng,date,startTime,awayTeam,awayTeamAbbr,homeTeam,homeTeamAbbr,awayTeamScore,homeTeamScore,venue,attendance,gameDuration
0,2019,"Wednesday, March 20, 2019",2019-03-20,5:35 p.m. ET,Seattle Mariners,SEA,Oakland Athletics,OAK,9,7,Tokyo Dome,45787,3:24
1,2019,"Thursday, March 21, 2019",2019-03-21,5:35 p.m. ET,Seattle Mariners,SEA,Oakland Athletics,OAK,5,4,Tokyo Dome,46451,4:27
2,2019,"Thursday, March 28, 2019",2019-03-28,4:10 p.m. ET,Pittsburgh Pirates,PIT,Cincinnati Reds,CIN,3,5,Great American Ball Park,44049,2:54
3,2019,"Thursday, March 28, 2019",2019-03-28,4:15 p.m. ET,Chicago White Sox,CHW,Kansas City Royals,KCR,3,5,Kauffman Stadium,31675,3:12
4,2019,"Thursday, March 28, 2019",2019-03-28,4:10 p.m. ET,Arizona D'Backs,ARI,Los Angeles Dodgers,LAD,5,12,Dodger Stadium,53086,2:49
5,2019,"Thursday, March 28, 2019",2019-03-28,4:10 p.m. ET,Colorado Rockies,COL,Miami Marlins,MIA,6,3,Marlins Park,25423,2:55
6,2019,"Thursday, March 28, 2019",2019-03-28,2:10 p.m. ET,St. Louis Cardinals,STL,Milwaukee Brewers,MIL,4,5,Miller Park,45304,2:36
7,2019,"Thursday, March 28, 2019",2019-03-28,4:10 p.m. ET,Cleveland Indians,CLE,Minnesota Twins,MIN,0,2,Target Field,39519,2:18
8,2019,"Thursday, March 28, 2019",2019-03-28,1:05 p.m. ET,Baltimore Orioles,BAL,New York Yankees,NYY,2,7,Yankee Stadium III,46928,3:06
9,2019,"Thursday, March 28, 2019",2019-03-28,4:07 p.m. ET,Los Angeles Angels,LAA,Oakland Athletics,OAK,0,4,Oakland-Alameda County Coliseum,22691,2:18


In [89]:
for divs in content('div')[:5]:
    h = divs.find('h3')
    games = divs('p', {'class':'game'})
    for p in games:
        a1 = p('a')[0]
        a2 = p('a')[1]
        
        boxscore = p.find('em')
        boxUrl = 'https://www.baseball-reference.com/' + boxscore.find('a')['href']
        
        boxRes = requests.get(boxUrl)
        boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')
        
        boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})
        print(boxUrl, 'StartTime: ', boxMeta('div')[1].text.split(': ')[1], int(boxMeta('div')[2].text.split(': ')[1].replace(',','')), boxMeta('div')[3].text.split(': ')[1], boxMeta('div')[4].text.split(': ')[1] )
        
        #print(boxscore.find('a')['href'])

https://www.baseball-reference.com//boxes/OAK/OAK201903200.shtml StartTime:  5:35 p.m. ET 45787 Tokyo Dome 3:24
https://www.baseball-reference.com//boxes/OAK/OAK201903210.shtml StartTime:  5:35 p.m. ET 46451 Tokyo Dome 4:27
https://www.baseball-reference.com//boxes/CIN/CIN201903280.shtml StartTime:  4:10 p.m. ET 44049 Great American Ball Park 2:54
https://www.baseball-reference.com//boxes/KCA/KCA201903280.shtml StartTime:  4:15 p.m. ET 31675 Kauffman Stadium 3:12
https://www.baseball-reference.com//boxes/LAN/LAN201903280.shtml StartTime:  4:10 p.m. ET 53086 Dodger Stadium 2:49
https://www.baseball-reference.com//boxes/MIA/MIA201903280.shtml StartTime:  4:10 p.m. ET 25423 Marlins Park 2:55
https://www.baseball-reference.com//boxes/MIL/MIL201903280.shtml StartTime:  2:10 p.m. ET 45304 Miller Park 2:36
https://www.baseball-reference.com//boxes/MIN/MIN201903280.shtml StartTime:  4:10 p.m. ET 39519 Target Field 2:18
https://www.baseball-reference.com//boxes/NYA/NYA201903280.shtml StartTime:

In [74]:
boxUrl = 'https://www.baseball-reference.com/' + boxscore.find('a')['href']

boxRes = requests.get(boxUrl)
boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')

boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})

boxMeta('div')[1].text

'Start Time: 5:35 p.m. ET'

In [92]:
boxUrl = 'https://www.baseball-reference.com/boxes/PIT/PIT200004040.shtml'
        
boxRes = requests.get(boxUrl)
boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')

boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})

In [118]:
for bm in boxMeta('div'):
    for bms in bm('strong'):
        print(bms.text.strip())
        
        if bms.text.strip() == 'Attendance':
            attendance = int(bm.text.split(': ')[1].replace(',',''))
        elif bms.text.strip() == 'Venue':
            venue = bm.text.split(': ')[1]
        elif bms.text.strip() == 'Game Duration':
            gameDur = bm.text.split(': ')[1]
    
  
print(attendance, venue, gameDur)     

Recheduled From
Attendance
Venue
Game Duration
54399 Three Rivers Stadium 2:51
